# Federated Learning Training Plan: Create Plan

Let's try to make protobuf-serializable Training Plan and Model that work after deserializing :)

Current list of problems:
 * No support for autograd in Plan tracing (.backward() doesn't work inside the Plan).
 * `tensor.shape` value seem to be recorded as constant during Plan tracing, so we need to pass `batch_size`, can't take it from tensor itself.
 * Plan needs a list of all Model params in the argument list, it would be nicer if this list is figured out automatically so you just pass the Model (not sure it's solvable jit might not accept the model as ScriptModule input?)
 * Plan doesn't return input args from Plan, e.g. if they were updated with inplace operation
 * others? 


In [1]:
%load_ext autoreload
%autoreload 2
import syft as sy
import torch as th
from torch import jit
from syft.serde import protobuf
import os
from syft.execution.state import State
from syft.execution.placeholder import PlaceHolder
from syft.execution.placeholder_id import PlaceholderId
from syft.execution.translation.torchscript import PlanTranslatorTorchscript 

sy.hook(globals())
# force protobuf serialization for tensors
hook.local_worker.framework = None
th.random.manual_seed(1)

C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Vova\AppData\Local\conda\conda\envs\pysyft\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Vova\AppData\Loc

Setting up Sandbox...
Done!


This utility function will serialize any object to protobuf binary and save to a file.

In [2]:
def serializeToBinPb(worker, obj, filename):
    pb = protobuf.serde._bufferize(worker, obj)
    bin = pb.SerializeToString()
    print("Writing %s to %s/%s" % (obj.__class__.__name__, os.getcwd(), filename))
    with open(filename, "wb") as f:
        f.write(bin)

## Step 1: Define the model

This model will train on MNIST data, it's very simple yet can demonstrate learning process.
There're 2 linear layers: 

* Linear 784x392
* ReLU
* Linear 392x10 

Not using nn.Module or nn.Linear for now, just vanilla class and tensors.
No autograd, gradients are hand-coded. 

As no loops supported inside Plan, 
we can't iterate over parameters so everything that works with params
(get/set, step) is moved into the model to make Plan code more generic.  

In [3]:
class Net():
    def __init__(self):
        super(Net, self).__init__()
        self.W1 = th.randn(392, 784) / th.sqrt(th.tensor(784.))
        self.b1 = th.zeros(392)
        self.W2 = th.randn(10, 392) / th.sqrt(th.tensor(392.))
        self.b2 = th.zeros(10)
        self.update_fn = None

    def forward(self, X):
        self.Z1 = X @ self.W1.t() + self.b1
        self.A1 = th.nn.functional.relu(self.Z1)
        self.Z2 = self.A1 @ self.W2.t() + self.b2
        return self.Z2

    def get_params(self):
        return self.W1, self.b1, self.W2, self.b2

    def set_params(self, *model_params):
        self.W1, self.b1, self.W2, self.b2 = model_params

    def grad(self, X, error):
        Z1_grad = (error @ self.W2) * (self.Z1 > 0).float()
        W1_grad = Z1_grad.t() @ X
        b1_grad = Z1_grad.sum(0)
        W2_grad = error.t() @ self.A1 
        b2_grad = error.sum(0)
        return W1_grad, b1_grad, W2_grad, b2_grad

model = Net()

## Step 2: Define Training Plan
### Loss function 
Batch size needs to be passed because otherwise `target.shape[0]` will be saved as `1` constant during Plan trace with dummy data.
Grad is also returned here. 

In [4]:
def cross_entropy_with_logits(output, target, batch_size):
    probs = th.nn.functional.softmax(output, dim=1)
    loss = -(target * th.log(probs)).mean()
    loss_grad = (probs - target) / (batch_size * target.shape[1])
    return probs, loss, loss_grad

### Optimization function
 
Just updates weights with grad*lr.

Note: can't do inplace update and return this value from the Plan, which is potentially bad for memory.

In [5]:
def naive_sgd(param, grad, **kwargs):
    return param - grad * kwargs['lr']

### Training Plan procedure

In [6]:
model_params = model.get_params()

# define plan input dimensions
X_size = (-1, 784)
y_size = (-1, 10)
scalar_size = (1,)
model_params_shapes = [p.shape for p in model_params]

args_shape = [
    X_size,  # X
    y_size,  # y
    scalar_size,  # batch_size
    scalar_size,  # lr
    *model_params_shapes  # *model_params
]

@sy.func2plan(args_shape=args_shape)
def training_plan(X, y, batch_size, lr, *model_params):
    # inject params into model
    model.set_params(*model_params)

    # forward pass
    output = model.forward(X)
    
    # loss
    probs, loss, loss_grad = cross_entropy_with_logits(output, y, batch_size)

    # backprop
    grads = model.grad(X, loss_grad)

    # step
    updated_params = [naive_sgd(param, grads[i], lr=lr) 
                      for i, param in enumerate(model_params)]
    
    # accuracy
    pred = th.argmax(probs, dim=1)
    target = th.argmax(y, dim=1)
    acc = pred.eq(target).float().sum() / batch_size

    return (
        loss,
        acc,
        *updated_params
    )

Let's look inside the Plan and print out the list of operations recorded.

In [7]:
print(training_plan)


<Plan training_plan id:48324959290 owner:me built>
def training_plan(_arg1, _arg2, _arg3, _arg4, _arg5, _arg6, _arg7, _arg8):
    _9 = _arg5.t()
    _10 = _arg1.__matmul__(_9)
    _11 = _10.__add__(_arg6)
    _12 = torch.nn.functional.relu(_11)
    _13 = _arg7.t()
    _14 = _12.__matmul__(_13)
    _15 = _14.__add__(_arg8)
    _16 = torch.nn.functional.softmax(_15, dim=1)
    _17 = torch.log(_16)
    _18 = _arg2.__mul__(_17)
    _19 = _18.mean()
    _out1 = _19.__neg__()
    _21 = _16.__sub__(_arg2)
    _22 = _arg3.__mul__(10)
    _23 = _21.__truediv__(_22)
    _24 = _23.__matmul__(_arg7)
    _25 = _11.__gt__(0)
    _26 = _25.float()
    _27 = _24.__mul__(_26)
    _28 = _27.t()
    _29 = _28.__matmul__(_arg1)
    _30 = _27.sum(0)
    _31 = _23.t()
    _32 = _31.__matmul__(_12)
    _33 = _23.sum(0)
    _34 = _29.__mul__(_arg4)
    _out2 = _arg5.__sub__(_34)
    _36 = _30.__mul__(_arg4)
    _out3 = _arg6.__sub__(_36)
    _38 = _32.__mul__(_arg4)
    _out4 = _arg7.__sub__(_38)
    _40 = _3

## Step 3: JIT Trace Training Plan

Note: Plan expects everything to be a tensor.


In [8]:
X_trace = th.randn(2, 784)
y_trace = th.randn(2, 10)
lr = th.tensor(0.001)
batch_size = th.tensor(32)

# Make torchscript plan
training_plan_torchscript = training_plan.translate_with(PlanTranslatorTorchscript)

# Let's see torchscript code
print(training_plan_torchscript.torchscript.code)

def forward(self,
    argument_1: Tensor,
    argument_2: Tensor,
    argument_3: Tensor,
    argument_4: Tensor,
    argument_5: Tensor,
    argument_6: Tensor,
    argument_7: Tensor,
    argument_8: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor, Tensor, Tensor]:
  _0 = torch.matmul(argument_1, torch.t(argument_5))
  _1 = torch.add(_0, argument_6, alpha=1)
  _2 = torch.relu(_1)
  _3 = torch.add(torch.matmul(_2, torch.t(argument_7)), argument_8, alpha=1)
  _4 = torch.softmax(_3, 1, None)
  _5 = torch.mean(torch.mul(argument_2, torch.log(_4)), dtype=None)
  _6 = torch.neg(_5)
  _7 = torch.div(torch.sub(_4, argument_2, alpha=1), torch.mul(argument_3, CONSTANTS.c0))
  _8 = torch.matmul(_7, argument_7)
  _9 = torch.to(torch.gt(_1, 0), 6, False, False, None)
  _10 = torch.mul(_8, _9)
  _11 = torch.matmul(torch.t(_10), argument_1)
  _12 = torch.sum(_10, [0], False, dtype=None)
  _13 = torch.matmul(torch.t(_7), _2)
  _14 = torch.sum(_7, [0], False, dtype=None)
  _15 = torch.sub(argument_5,

## Step 4: Serialize!

Now it's time to serialize model params and plans to protobuf and save them for further usage:
 * In "Execute Plan" notebook, we load and execute these plans & model, from Python.
 * In "Host Plan" notebook, we send these plans & model to PyGrid, so it can be executed from other worker (e.g. syft.js).

**NOTE:**
 * We don't serialize full Model, only weights. How the Model is serialized is TBD.
   State is suitable protobuf class to wrap list of Model params tensors.

In [9]:
serializeToBinPb(hook.local_worker, training_plan, "tp_ops.pb")
serializeToBinPb(hook.local_worker, training_plan_torchscript, "tp_ts.pb")

# wrap weights in State to serialize
model_params_state = State(
    owner=hook.local_worker,
    state_placeholders=[PlaceHolder().instantiate(param) for param in model_params]
)

serializeToBinPb(hook.local_worker, model_params_state, "model_params.pb")


Writing Plan to e:\ml/tp_ops.pb
Writing Plan to e:\ml/tp_ts.pb
Writing State to e:\ml/model_params.pb
